# Finetune DistilBERT on the WNUT 17 dataset to detect new entities.
This shared task focuses on identifying unusual, previously-unseen entities in the context of emerging discussions. Named entities form the basis of many modern approaches to other tasks (like event clustering and summarisation), but recall on them is a real problem in noisy text - even among annotators. This drop tends to be due to novel entities and surface forms. Take for example the tweet “so.. kktny in 30 mins?” - even human experts find entity kktny hard to detect and resolve. This task will evaluate the ability to detect and classify novel, emerging, singleton named entities in noisy text.

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from datasets import load_dataset

wnut = load_dataset("wnut_17", trust_remote_code=True)

Generating train split:   0%|          | 0/3394 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [5]:
wnut["train"][55]

{'id': '55',
 'tokens': ['There',
  "'s",
  'a',
  'lot',
  'of',
  'people',
  'showing',
  'off',
  'their',
  'iPhones',
  'on',
  'facebook',
  'today',
  ',',
  'so',
  '&lt;',
  'so',
  'is',
  'at',
  'such',
  'a',
  'place',
  ',',
  'it',
  "'s",
  'really',
  'not',
  'that',
  'interesting',
  ';o',
  ')'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  11,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [13]:
label_list = wnut["train"].features[f"ner_tags"].feature.names

In [6]:
# The next step is to load a DistilBERT tokenizer to preprocess the tokens field:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [7]:
# Example for is_split_into_words = True
example = wnut["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 '@',
 'paul',
 '##walk',
 'it',
 "'",
 's',
 'the',
 'view',
 'from',
 'where',
 'i',
 "'",
 'm',
 'living',
 'for',
 'two',
 'weeks',
 '.',
 'empire',
 'state',
 'building',
 '=',
 'es',
 '##b',
 '.',
 'pretty',
 'bad',
 'storm',
 'here',
 'last',
 'evening',
 '.',
 '[SEP]']

In [8]:
def tokenize_and_align_labels(examples):
    # Tokenize the 'tokens' field; each example is a list of words (not a string)
    tokenized_inputs = tokenizer(
        examples["tokens"], 
        truncation=True, 
        is_split_into_words=True
    )

    labels = []  # This will store the new label lists, aligned with tokenized subwords

    # Loop through each example in the batch
    for i, label in enumerate(examples["ner_tags"]):
        # Get a list that maps each tokenized token to its original word index
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None  # Keep track of the previous word index
        label_ids = []  # This will store the aligned labels for this example

        # Loop through each token's word index
        for word_idx in word_ids:
            if word_idx is None:
                # Special tokens (like [CLS], [SEP]) get a label of -100 (ignored in loss)
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                # First token of a word gets the real label for that word
                label_ids.append(label[word_idx])
            else:
                # Subsequent tokens (subwords) of the same word get -100
                label_ids.append(-100)
            previous_word_idx = word_idx  # Update for next token

        labels.append(label_ids)  # Add the aligned labels for this example

    # Add the new labels to the tokenized inputs
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [9]:
# apply the preprocessing function over the entire dataset

tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3394 [00:00<?, ? examples/s]

Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [10]:
# create a batch of examples using DataCollatorWithPadding.

from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [11]:
# Evaluate

import evaluate

seqeval = evaluate.load("seqeval")

In [14]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [15]:
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
}
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}

In [16]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=13, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
training_args = TrainingArguments(
    output_dir="my_awesome_wnut_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.283819,0.522199,0.228916,0.318299,0.937925
2,No log,0.272966,0.617594,0.318814,0.420538,0.942799


/home/proxi/anaconda3/envs/pytorch_env/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=426, training_loss=0.20931317324929394, metrics={'train_runtime': 34.559, 'train_samples_per_second': 196.418, 'train_steps_per_second': 12.327, 'total_flos': 91781128898820.0, 'train_loss': 0.20931317324929394, 'epoch': 2.0})

In [18]:
# Inference

In [19]:
text = "The Golden State Warriors are an American professional basketball team based in San Francisco."


In [21]:
from transformers import pipeline

classifier = pipeline("ner", model="Proxiii/my_awesome_wnut_model")
classifier(text)

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/266M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


[{'entity': 'B-location',
  'score': np.float32(0.3684948),
  'index': 2,
  'word': 'golden',
  'start': 4,
  'end': 10},
 {'entity': 'B-location',
  'score': np.float32(0.2616857),
  'index': 3,
  'word': 'state',
  'start': 11,
  'end': 16},
 {'entity': 'B-location',
  'score': np.float32(0.46829382),
  'index': 13,
  'word': 'san',
  'start': 80,
  'end': 83},
 {'entity': 'B-location',
  'score': np.float32(0.46683404),
  'index': 14,
  'word': 'francisco',
  'start': 84,
  'end': 93}]